Imports

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib

path to source

In [0]:
path = '/content/drive/My Drive/DATA/10.txt'

In [0]:
text = open(path,'r').read()

Break out vocab

In [0]:
vocab = sorted(set(text))

index letters

In [0]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [0]:
ind_to_char = np.array(vocab)

In [0]:
encoded_txt = np.array([char_to_ind[c] for c in text])

check source

In [8]:
print(text[1000:2000])

 BIBLE ***














The Old Testament of the King James Version of the Bible




The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day.

1:6 And God said, Let there be a firmament in the midst of the waters,
and let it divide the waters from the waters.

1:7 And God made the firmament, and divided the waters which were
under the firmament from the waters which were above the firmament:
and it was so.

1:8 And God called the firmament Heaven. And the evening and the
morning were the second day.

1:9 And God said, Let the wa

In [0]:
lines = '''1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.'''



In [10]:
len(lines)

146

format for training

In [0]:
seq_len = 146

In [0]:
total_num_seq = len(text) // (seq_len+1)

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_txt)

In [0]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [0]:
def create_seq_targets(seq):
  input_txt = seq[:-1]
  target_txt = seq[1:]
  return input_txt,target_txt

In [0]:
dataset = sequences.map(create_seq_targets)

In [17]:
for input_txt,target_txt in dataset.take(1):
  print(input_txt.numpy())
  print("".join(ind_to_char[input_txt.numpy()]))
  print('\n')
  print(target_txt.numpy())
  print("".join(ind_to_char[target_txt.numpy()]))

[48 64 61  1 44 74 71 66 61 59 76  1 35 77 76 61 70 58 61 74 63  1 33 30
 71 71 67  1 71 62  1 48 64 61  1 39 65 70 63  1 38 57 69 61 75  1 30 65
 58 68 61  0  0  0 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10  0 33 30 43 43 39  1  8  4 16 15  9  1 51 29 47  1 43 42 33
  1 43]
The Project Gutenberg EBook of The King James Bible


**********************************************************************
EBOOK (#10) WAS ONE O


[64 61  1 44 74 71 66 61 59 76  1 35 77 76 61 70 58 61 74 63  1 33 30 71
 71 67  1 71 62  1 48 64 61  1 39 65 70 63  1 38 57 69 61 75  1 30 65 58
 68 61  0  0  0 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10  0 33 30 43 43 39  1  8  4 16 

In [18]:
len(text)

4352315

In [0]:
batch_size =150

In [0]:
buffer_size = 10000


dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [0]:
vocab_size = len(vocab)

In [0]:
embed_dim = 64

prepare model

In [0]:
rnn_neurons = 1026


In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [0]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [0]:
from tensorflow.keras.models import Sequential

In [0]:
from tensorflow.keras.layers import Embedding,GRU,Dense,LSTM

In [0]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):

  model = Sequential()
  model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
  model.add(GRU(rnn_neurons,return_sequences=True,
                stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  model.compile('adam',loss=sparse_cat_loss)

  return model

model = create_model(vocab_size=vocab_size,embed_dim=embed_dim,rnn_neurons=rnn_neurons,batch_size=batch_size)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (150, None, 64)           5312      
_________________________________________________________________
gru (GRU)                    (150, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (150, None, 83)           85241     
Total params: 3,451,729
Trainable params: 3,451,729
Non-trainable params: 0
_________________________________________________________________


run model

In [0]:
epochs = 5

In [0]:
model.fit(dataset,epochs=epochs)

save model and tools

In [0]:
joblib.dump(ind_to_char,"ind_to_char.pkl")
joblib.dump(char_to_ind,"char_to_ind.pkl")
model.save("model.h5")
